In [1]:
import pandas as pd
import glob
from tqdm import tqdm
import re
import os
import numpy as np
from pyfaidx import Fasta
from pysam import VariantFile
import gc

## Generating the SNP inputs

From the clair3 re_run directory (`/hpcfs/groups/phoenix-hpc-avsci/Callum_MacPhillamy/clair3_rerun/sample{01..20}.vcf.gz`) 
use the below command to collect all the VCFs for each sample.

```bash
for sample in $(seq -w 1 20)
do
    find . -path "*sample${sample}/*/merge_output.vcf.gz" | sort -V > "sample${sample}.files"
done
```

This will create a file for each sample with the paths to the VCFs.

We can then merge them with `bcftools` and create a single VCF for each sample.

Here, `$f` is the file we created in the previous step.

```bash
for f in *.files
do
    bcftools concat --threads 2 -f $f -O z -o $(basename -s .files $f).vcf.gz
done
```

In [2]:
# Load the Illumina manifest
manifest = pd.read_csv('/Users/callummacphillamy/Projects/REFERENCES/UMD3.1.1/BovineSNP50_v3_A2.csv.gz',
                       skiprows=7, header=0)

# Subset to just the SNP name and RefStrand
# subset_manifest = manifest[['Name', 'RefStrand']].copy()
manifest.head()

# Load the dbSNP vcf
dbsnp_dict = {}
dbsnp = VariantFile('/Users/callummacphillamy/Projects/REFERENCES/ARS_UCD1.2/9913_GCA_002263795.2_current_ids.vcf.gz')
for record in tqdm(dbsnp.fetch()):
    chrpos = f'{record.chrom}_{record.pos}'
    ref = record.ref

    # Skip if there are multiple alts
    if len(record.alts) != 1:
        continue
    alt = record.alts[0]
    dbsnp_dict[chrpos] = {'ref': ref, 'alt': alt}
print(len(dbsnp_dict))

dbsnp_df = pd.DataFrame.from_dict(dbsnp_dict, orient='index').reset_index()

manifest_clean = manifest[['Name', 'SNP', 'Chr', 'MapInfo']].copy()
manifest_clean = manifest_clean[~manifest_clean['Chr'].isna()]
manifest_clean = manifest_clean[~manifest_clean['MapInfo'].isna()]
manifest_clean['MapInfo'] = manifest_clean['MapInfo'].astype(int)
manifest_clean

[E::idx_find_and_load] Could not retrieve index file for '/Users/callummacphillamy/Projects/REFERENCES/ARS_UCD1.2/9913_GCA_002263795.2_current_ids.vcf.gz'
106807756it [06:39, 267341.71it/s]


101997050


,Name,SNP,Chr,MapInfo
0,ABCA12,[A/G],2,103548215
1,APAF1,[T/C],5,63150400
2,ARS-BFGL-BAC-10172,[A/G],14,6371334
3,ARS-BFGL-BAC-1020,[T/C],14,7928189
4,ARS-BFGL-BAC-10245,[T/C],14,31819743
...,...,...,...,...
53213,UA-IFASA-9774,[T/C],14,39201271
53214,UA-IFASA-9790,[T/C],19,56816549
53215,UA-IFASA-9812,[T/C],29,48012818
53216,UA-IFASA-9813,[T/C],19,32508700


In [4]:

versa_ars = pd.read_csv('../SNP_analyses/versa50K.arsucd.bed.gz', sep='\t',
                          header=None,
                          names=['chrom','pos-1','pos','name','SNP','strand'])

versa_ars['ars_ChrPos'] = versa_ars['chrom'].astype(str) + '_' + versa_ars['pos'].astype(str)
versa_ars['SNP'] = versa_ars['SNP'].str.strip('[]')
versa_ars['ilmn_ref'] = versa_ars['SNP'].str.split('/').str[0]
versa_ars['ilmn_alt'] = versa_ars['SNP'].str.split('/').str[1]
print(versa_ars.shape)
# versa_ars = versa_ars.merge(subset_manifest, left_on='name', right_on='Name', how='left')
print(versa_ars.shape)

(52034, 9)
(52034, 9)


In [5]:
# Load the original SNP dataframe. I.e. Joe's genotypes on Versa50K
og_snps = pd.read_parquet('../SNP_analyses/genotypes/3DG_Nucleus_array_genotypes.parquet')
og_snps

,SNP,ChrPos,Chr,Pos,3DWFQ0404,3DWFQ0459,3DWFN0018,3DWFP0413,3DWFQ0177,3DWF22T0070,...,3DWF22T0549,3DWFR0281,3DWFS0069,3DWFS0202,3DWFP0398,3DWFP0416,3DWFQ0389,3DWFR0043,WGWFJ0085,3DWFR0261
5142,BovineHD0100000005,1_16947,1,16947,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5143,chr3_134833,1_19554,1,19554,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5144,BovineHD0100000033,1_111645,1,111645,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5145,BovineHD0100000035,1_120183,1,120183,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5146,Hapmap43437-BTA-101873,1_135098,1,135098,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135739,BovineHD3200000384,MT_15308,MT,15308,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
135740,BovineHD3200000389,MT_15505,MT,15505,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135741,BovineHD3200000395,MT_15894,MT,15894,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135742,BovineHD3200000406,MT_16231,MT,16231,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [6]:
versa_ars_umd_genotypes = pd.merge(versa_ars[['name','ars_ChrPos','ilmn_ref','ilmn_alt']], og_snps, left_on='name',
                               right_on='SNP',
                               how='left')
versa_ars_umd_genotypes

# Map the floats back to the actual alleles
float_to_allele_dict = {}
for row in versa_ars_umd_genotypes.itertuples():
    chrpos = row.ars_ChrPos
    float_to_allele_dict[chrpos] = {0. : f'{row.ilmn_ref}{row.ilmn_ref}',
                                    1. : f'{row.ilmn_ref}{row.ilmn_alt}',
                                    2. : f'{row.ilmn_alt}{row.ilmn_alt}',
                                    np.nan: '--'}

ars_genotype_dict = {}
for sample in versa_ars_umd_genotypes.columns[8:]:
    ars_genotype_dict[sample] = {}
    for row in versa_ars_umd_genotypes.loc[:, ['ars_ChrPos', sample]].itertuples():
        chrpos = row.ars_ChrPos
        ars_genotype_dict[sample][chrpos] = float_to_allele_dict[chrpos].get(row[2], '--')
        

ars_genotype_df = pd.DataFrame.from_dict(ars_genotype_dict, orient='index').T
ars_genotype_df = ars_genotype_df.rename_axis('ars_ChrPos').reset_index()
ars_genotype_df

chrpos_set = set(versa_ars_umd_genotypes['ars_ChrPos'].values)
ars_genotype_df

,ars_ChrPos,3DWFQ0404,3DWFQ0459,3DWFN0018,3DWFP0413,3DWFQ0177,3DWF22T0070,3DWFP0355,WGWFL0149,3DWFS0137,...,3DWF22T0549,3DWFR0281,3DWFS0069,3DWFS0202,3DWFP0398,3DWFP0416,3DWFQ0389,3DWFR0043,WGWFJ0085,3DWFR0261
0,1_752778,AA,AA,AA,AA,AA,AA,AA,--,AA,...,AA,--,AA,AA,AA,AA,AA,AA,AA,AA
1,1_776231,TT,TT,TT,TT,TT,TT,TT,TT,TT,...,TT,TT,TT,TT,TT,TT,TT,TT,TT,TT
2,1_907810,CC,CC,CC,CC,CC,CC,CC,CC,CC,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
3,1_1032564,GG,GG,GG,GG,GG,GG,GG,GG,GG,...,GG,GG,GG,GG,GG,GG,GG,GG,GG,GG
4,1_1110393,AA,AG,AG,AA,AA,AA,AG,AA,AA,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50042,Y_23289148,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
50043,Y_24113061,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
50044,Y_24881352,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
50045,Y_25004275,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--


In [7]:
sample_dict = {
    'sample01':'3DWFQ0404',
    'sample02':'3DWFQ0459',
    'sample03':'3DWFN0018',
    'sample04':'3DWFP0413',
    'sample05':'3DWFQ0177',
    'sample06':'3DWF22T0070',
    'sample07':'3DWFP0355',
    'sample08':'WGWFL0149',
    'sample09':'3DWFS0137',
    'sample10':'3DWFR0017',
    'sample11':'3DWF22T0415',
    'sample12':'3DWF22T0240',
    'sample13':'3DWFQ0458',
    'sample14':'3DWFS0123',
    'sample15':'3DWF22T0036',
    'sample16':'3DWF22T0549',
    'sample17':'3DWFR0281',
    'sample18':'3DWFS0069',
    'sample19':'3DWFS0202',
    'sample20':'3DWFP0398'
}

sample = pd.read_csv(
    '/Users/callummacphillamy/Projects/tuwa_manuscript/TuWa_manuscript/SNP_analyses/genotypes/clair3_rerun/sample01.vcf.gz',
    sep='\t', comment='#', header=None, dtype={0:str},
    names=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT','sample'])

clean_sample = sample[sample['FILTER'] != 'LowQual']

# Remove indels/ Only consider SNPs
clean_sample = clean_sample[clean_sample['REF'].str.len() == 1]
clean_sample = clean_sample[clean_sample['ALT'].str.len() == 1]
clean_sample = clean_sample.copy()


clean_sample['ChrPos'] = clean_sample['CHROM'].astype(str) + '_' + clean_sample['POS'].astype(str)
clean_sample = clean_sample[['ChrPos','REF','ALT','sample']]
clean_sample.copy()

clean_sample['sample'] = clean_sample['sample'].str.split(':').str[0]

# Turn the sample column into the alleles genotype. E.g. 0/1 -> AT
def get_genotype(row):
    genotype = row['sample']
    ref = row['REF']
    alt = row['ALT']
    if genotype == '0/0':
        return ref + ref
    elif genotype == '0/1':
        return ref + alt
    elif genotype == '1/1':
        return alt + alt
    else:
        return '--'

# Apply the function to the dataframe
clean_sample['genotype'] = clean_sample.apply(get_genotype, axis=1)

clean_sample

,ChrPos,REF,ALT,sample,genotype
0,1_25784,A,T,0/1,AT
1,1_25787,A,G,0/1,AG
2,1_25797,C,T,0/1,CT
3,1_25810,C,G,0/1,CG
4,1_25828,C,A,0/1,CA
...,...,...,...,...,...
9019576,Y_59471086,A,G,0/1,AG
9019577,Y_59471104,C,G,0/1,CG
9019578,Y_59471116,C,G,0/1,CG
9019588,Y_59474743,G,A,0/1,GA


In [8]:
# We will use the DBSNP to resolve what the ref and alt alleles should be
sample_to_compare = sample_dict['sample01']
ars_genotype_df[['ars_ChrPos',sample_to_compare]]
comparison = pd.merge(ars_genotype_df[['ars_ChrPos',sample_to_compare]], clean_sample[['ChrPos','genotype']], left_on='ars_ChrPos',
                       right_on='ChrPos',
                       how='inner')

comparison

,ars_ChrPos,3DWFQ0404,ChrPos,genotype
0,1_1110393,AA,1_1110393,TT
1,1_1291908,CC,1_1291908,CC
2,1_1350801,AG,1_1350801,GA
3,1_1566539,CC,1_1566539,GG
4,1_1604940,CC,1_1604940,CC
...,...,...,...,...
26755,Y_490228,AG,Y_490228,AG
26756,Y_549318,AG,Y_549318,CT
26757,Y_614721,AG,Y_614721,AG
26758,Y_780515,TC,Y_780515,AG


In [9]:



big_comparison = comparison.merge(dbsnp_df, left_on='ChrPos', right_on='index', how='left')
big_comparison

# Compare the genotypes from the two samples, relative to the dbSNP ref and alt
correct = 0
incorrect = []

complementary = {'AA':'TT',
                 'CC':'GG',
                 'TT':'AA',
                 'GG':'CC',
                 'AC':'TG',
                 'TG':'AC',
                 'TC':'AG',
                 'AG':'TC'}


missing = 0

for row in tqdm(big_comparison.itertuples()):

    # Skip if there are any NaNs in the row
    if sum([pd.isna(val) for val in row]) > 0:
        continue
        
    true_ref = row.ref
    true_alt = row.alt

    true_hom_ref = true_ref + true_ref
    true_het = true_ref + true_alt
    true_het_other_way = true_alt + true_ref
    true_hom_alt = true_alt + true_alt

    heterozygous = [true_het, true_het_other_way]
    

    ont_gt = row.genotype
    array_gt = row[2]

    if ont_gt == '--' or array_gt == '--':
        missing += 1
        continue

    # Case 1: Both genotypes match the true homozygous ref
    if ont_gt == true_hom_ref and array_gt == true_hom_ref:
        correct += 1
    
    # Case 2: Both genotypes match the true heterozygous
    elif ont_gt in heterozygous and array_gt in heterozygous:
        correct += 1

    # Case 3: Both genotypes match the true homozygous alt
    elif ont_gt == true_hom_alt and array_gt == true_hom_alt:
        correct += 1

    # Case 4: Array complement matches the true homozygous ref
    elif complementary.get(array_gt, '') == true_hom_ref and ont_gt == true_hom_ref:
        correct += 1
    
    # Case 5: Array complement matches the true homozygous alt
    elif complementary.get(array_gt, '') in true_hom_alt and ont_gt in true_hom_alt:
        correct += 1
    
    # Case 6: Array is heterozygous and complementary to the true heterozygous
    elif complementary.get(array_gt, '') in heterozygous and ont_gt in heterozygous:
        correct += 1
    
    # Case 7: Mismatch
    else:
        incorrect.append(row)
    
    
print(f'Correct: {correct}')
print(f'Incorrect: {len(incorrect)}')
print(f'Correct: {correct/(correct + len(incorrect))}')

    

26760it [00:00, 311032.71it/s]

Correct: 25764
Incorrect: 687
Correct: 0.9740274469774299


In [ ]:
print(incorrect)

In [ ]:
sample_to_compare = sample_dict['sample01']
comparison = pd.merge(ars_genotype_df[['ars_ChrPos',sample_to_compare]], clean_sample[['ChrPos','genotype']], left_on='ars_ChrPos',
                       right_on='ChrPos',
                       how='inner')
comparison

# correct if complement means TC and AG/GA are considered the same, for example.

complement_dict = {'A':'T',
                   'T':'A',
                   'C':'G',
                   'G':'C'}

def complement(seq):
    """
    Returns the complement of a DNA sequence.
    """
    return ''.join(complement_dict.get(base, base) for base in seq)


result = {'correct_as_is':0,
          'correct_if_complement':0,
          'incorrect':[]}
for row in tqdm(comparison.itertuples()):
    ont = row.genotype
    ars = row[2]

    ont = ''.join(sorted(ont))
    ars = ''.join(sorted(ars))

    if '--' in [ont, ars]:
        continue

    if ont == ars:
        result['correct_as_is'] += 1
    elif ont == complement(ars):
        result['correct_if_complement'] += 1
    else:
        result['incorrect'].append([row.ars_ChrPos, ont, ars])
print(result['correct_as_is'])
print(result['correct_if_complement'])
print(len(result['incorrect']))
print('Total potentially correct: ', result['correct_as_is'] + result['correct_if_complement'])
print('Total examined: ', result['correct_as_is'] + result['correct_if_complement'] + len(result['incorrect']))
print('Percentage correct: ', (result['correct_as_is'] + result['correct_if_complement']) / (result['correct_as_is'] + result['correct_if_complement'] + len(result['incorrect'])) * 100)

In [ ]:
def get_genotype(gt_float, ref, alt, comp_dict, use_comp=False):
    if gt_float == 0.:
        if use_comp:
            return comp_dict[ref]+comp_dict[ref]
        else:
            return ref+ref
    elif gt_float == 1.:
        if use_comp:
            return comp_dict[ref]+comp_dict[alt]
        else:
            return ref+alt
    elif gt_float == 2.:
        if use_comp:
            return comp_dict[alt]+comp_dict[alt]
        else:
            return alt+alt
    else:
        return np.nan

In [ ]:
correct = 0
half_correct = 0
incorrect = 0
ont_missing = 0
ilmn_missing = 0
valid = 0
for row in tqdm(comparison.itertuples()):
    if len(row.REF) > 1 or len(row.ALT) > 1:
        continue
    if row.sample not in [1.,2.,0.] and row._9 not in [1.,2.,0.]:
        ont_missing += 1
        ilmn_missing += 1
        continue
    elif row.sample not in [1.,2.,0.]:
        ont_missing += 1
        continue
    elif row._9 not in [1.,2.,0.]:
        ilmn_missing += 1
        continue
    elif row.ilmn_ref in ['I','D']:
        continue
    
    # print(row)
    ont_alleles = [row.REF, row.ALT]
    ilmn_alleles = [row.ilmn_ref, row.ilmn_alt]
    ont_gt = get_genotype(row.sample, row.REF, row.ALT, comp_dict=comp_dict, use_comp=False)
    comp_ont_gt = get_genotype(gt_float = row.sample,
                               ref = row.REF,
                               alt = row.ALT, comp_dict=comp_dict, use_comp=True)
    ilmn_gt = get_genotype(row._9, row.ilmn_ref, row.ilmn_alt, comp_dict=comp_dict, use_comp=False)

    if ont_gt is np.nan or ilmn_gt is np.nan:
        print(row)
        raise ValueError('genotype is nan')

    if ont_gt == ilmn_gt:
        correct += 1
    
    elif ont_gt[0] in ilmn_gt or ont_gt[1] in ilmn_gt:
        half_correct += 1
    
    elif ont_gt != ilmn_gt:

        # Try to correct the genotype
        if comp_ont_gt == ilmn_gt:
            correct += 1
        elif comp_ont_gt[0] in ilmn_gt or comp_ont_gt[1] in ilmn_gt:
            half_correct += 1
        else:
            incorrect += 1
    
    valid += 1

print('Of {} valid SNPs:'.format(valid))
print(correct, np.around(correct / valid, decimals=2), 'proportion were concordant')
print(half_correct, np.around(half_correct / valid, decimals=2), 'proportion were half concordant')
print(incorrect, np.around(incorrect / valid,decimals=2), 'proportion were discordant')

In [ ]:

for row in comparison.itertuples():
    if row.ALT == '.':
        if row.REF == row.ilmn_ref:
            if row.sample == row._8:
                correct += 1
            elif np.abs(row.sample - row._8) == 1:
                half_correct += 1
            else:
                incorrect += 1
    
    # REF and ALT match Illumina REF and ALT
    if row.REF == row.ilmn_ref and row.ALT == row.ilmn_alt:
        if row.sample == row._8:
            correct += 1
        elif np.abs(row.sample - row._8) == 1:
            half_correct += 1
        else:
            incorrect += 1
    
    # REF and ALT don't match Illumina REF and ALT
    elif row.REF != row.ilmn_ref or row.ALT != row.ilmn_alt:
        
        # See if using the complement fixes it.
        if row.REF == comp_dict[row.ilmn_ref] and row.ALT == comp_dict[row.ilmn_alt]:
            if row.sample == row._8:
                correct += 1
            elif np.abs(row.sample - row._8) == 1:
                half_correct += 1
            else:
                incorrect += 1
        
        
        if row.sample == row._8:
            incorrect += 1
        elif np.abs(row.sample - row._8) == 1:
            half_correct += 1
        else:
            correct += 1


In [ ]:
%%bash
#for i in {1..29}
#do
#    grep -w "chr${i}" /Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/SNP_analyses/SNP_bed_files/versa50K_SNPs.liftover.bed > /Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/SNP_analyses/versa50K_SNPs.chr${i}.bed
#done
#grep "chrX" /Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/SNP_analyses/SNP_bed_files/versa50K_SNPs.liftover.bed > /Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/SNP_analyses/versa50K_SNPs.chrX.bed
#grep "chrY" /Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/SNP_analyses/SNP_bed_files/versa50K_SNPs.liftover.bed > /Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/SNP_analyses/versa50K_SNPs.chrY.bed

In [ ]:
#`~/software/gatk-4.4.0.0/gatk GenotypeGVCFs -R ~/Projects/REFERENCES/bosTau6/bosTau6.fa -V merge_output.gvcf.gz -O test.vcf.gz -L test.bed -all-sites`
# files = glob.glob('../RESULTS/mapped_to_bosTau6/SNPs/*/*/*.gvcf.gz')
files = glob.glob('../RESULTS/mapped_to_bosTau6/SNPs/sample06/*/*.gvcf.gz')
regions = '../SNP_analyses/SNP_bed_files'
for file in files:
    sample = file.split('/')[-3]
    
    if sample not in file:
        continue
    chromosome = file.split('/')[-2]
    region = os.path.join(regions, f'versa50K_SNPs.{chromosome}.bed')
    vcf = os.path.join("../SNP_analyses/genotypes/mapped_to_bosTau6",
                       f"{sample}.{chromosome}.versa50K.vcf.gz")
    ! ~/software/gatk-4.4.0.0/gatk GenotypeGVCFs -R ~/Projects/REFERENCES/bosTau6/bosTau6_w_wagyuY.fa -V {file} -O {vcf} -L {region} -all-sites
    
## There seems to be some issue with the length of chromosome 3 in the reference genome, vs what's in the bed file...
## I think the best solution will be to drop those SNPs and hope it's not too many...

In [ ]:
# samples = ['01','03','05','12']
samples = ['06']
for sample in samples:
    ! bcftools concat -O z ../SNP_analyses/genotypes/mapped_to_bosTau6/sample{sample}.chr*.vcf.gz | bcftools sort -O z > ../SNP_analyses/genotypes/mapped_to_bosTau6/sample{sample}.versa50K.vcf.gz

In [ ]:
DBSNP = '/Users/callummacphillamy/Projects/REFERENCES/bosTau6/Bos_taurus.vcf'
dbsnp = pd.read_csv(DBSNP, sep="\t", comment="#", dtype={0:str}, header=None)
dbsnp.columns = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO']
dbsnp = dbsnp.copy()
dbsnp = dbsnp[dbsnp['REF'].isin(['A','C','G','T'])]

dbsnp_dict_df = {'ChrPos':[],
         'Ref':[],
         'Alt':[],
         'ID':[]}
dbsnp_dict = {}
for row in tqdm(dbsnp.itertuples()):
    #print(row)
    info = row.INFO.split(';')
    type = info[1].split('=')[1]
    if type != 'SNV':
        continue
    chrpos = 'chr' + str(row.CHROM) + '_' + str(row.POS)
    alt = row.REF
    ref = row.ALT
    id = row.ID
    dbsnp_dict_df['ChrPos'].append(chrpos)
    dbsnp_dict_df['Ref'].append(ref)
    dbsnp_dict_df['Alt'].append(alt)
    dbsnp_dict_df['ID'].append(id)
    dbsnp_dict[chrpos] = {'Ref':ref,
                            'Alt':alt,
                            'ID':id}
del dbsnp
gc.collect()

In [ ]:
def load_vcf(vcf, sample):
    vcf = pd.read_csv(vcf, sep='\t', comment='#', header=None)
    vcf.columns = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT',sample]
    return vcf

unambiguous_snps = {'AC': ('A', 'C', 'TOP'),
                        'AG': ('A', 'G', 'TOP'),
                        'TC': ('T', 'C', 'BOT'),
                        'TG': ('T', 'G', 'BOT'),
                        'CA': ('A', 'C', 'TOP'),
                        'GA': ('A', 'G', 'TOP'),
                        'CT': ('T', 'C', 'BOT'),
                        'GT': ('T', 'G', 'BOT')}

def get_sequence(reference=None, snp_pos=None, ref_allele=None):
    """
    Get the sequence around a SNP position.
    Args:
        reference: A pyfaidx Fasta instance
        snp_pos: a tuple with form (chr, pos)
        ref_allele: the ref allele from the VCF file

    Returns:
        sequence: str, the sequence around the SNP position
    """


    complementary_bases = {'A':'T',
                           'T':'A',
                           'C':'G',
                           'G':'C'}

    if reference is None:
        raise ValueError('Reference should be a pyfaidx.Fasta instance')
    if snp_pos is None:
        raise ValueError('snp_pos should be a tuple with form (chr, pos)')


    # Check nucleotide at chr pos matches the ref allele
    ref_position = reference[str(snp_pos[0])][snp_pos[1] - 1: snp_pos[1]]
    print(ref_position)

    if ref_allele.upper() != str(ref_position).upper() and ref_allele.upper() != complementary_bases[str(ref_position).upper()]:
        print(f'Ref allele: {ref_allele}, Allele at position: {ref_position}')
        raise ValueError('The nucleotide at the provided position does not match'
                         ' the reference allele')


    sequence = str(reference[str(snp_pos[0])][snp_pos[1] - 41 : snp_pos[1] + 40])
    if sequence[40:41].upper() != ref_allele.upper() and sequence[40:41].upper() != complementary_bases[ref_allele.upper()]:
        raise ValueError('Midpoint of sequence doesn\'t match the ref allele')

    return sequence.upper()

def resolve_ambiguous_snp(seq=None, ref_allele=None, alt_allele=None):
    """
    Resolve ambiguous SNPs (A/T and C/G) using the sequence walk method.

    Args:
        seq: str, sequence around the SNP. The SNP should be in the center of 
            the sequence with an equal number of nucleotides up and down stream.
        ref_allele: str, The reference allele from the VCF file
        alt_allele: str, the alternate allele from the VCF file

    Returns:
        allele_a, allele_b, strand - The reference allele, alternate allele and strand

    Examples:
        >>> resolve_ambiguous_snp(seq="ACGGGGACAGATATGTTAACT", ref_allele='A', alt_allele='T')
        ('T', 'A', 'BOT')

    Raises:
        ValueError: If the ref allele does not match the nucleotide at the midpoint of the sequence
        ValueError: If the strand of the SNP cannot be resolved
    """

    _unambiguous_snps = {'AC': None,
                         'AG': None,
                         'TC': None,
                         'TG': None,
                         'CA': None,
                         'GA': None,
                         'CT': None,
                         'GT': None}

    # complementary_bases = {'A':'T',
    #                        'T':'A',
    #                        'C':'G',
    #                        'G':'C'}

    if len(seq) % 2 == 0:
        raise ValueError("Sequence length is even. This will cause issues.")

    seq_midpoint = len(seq) // 2

    if seq[seq_midpoint] != ref_allele and seq[seq_midpoint] != alt_allele: #and seq[seq_midpoint] != complementary_bases[ref_allele]:
        print('Neither alleles match the midpoint')
        print(f"Ref allele: {ref_allele}, Allele at midpoint: {seq[seq_midpoint]}")
        raise ValueError("Ref allele does not match the expected nucleotide at midpoint")

    # Begin sequence walk to resolve strand and A B alleles
    walk_toward_5_prime = [i for i in range(seq_midpoint - 1, 0-1, -1)]
    walk_toward_3_prime = [i for i in range(seq_midpoint + 1, len(seq))]

    walking_nt_positions = zip(walk_toward_5_prime, walk_toward_3_prime)

    if ref_allele is None:
        raise TypeError("ref_allele should not be None")

    # if alt_allele is None:
    #     alt_allele = ref_allele

    for i, j in walking_nt_positions:
        _5_prime_nt = seq[i]
        _3_prime_nt = seq[j]
        _pairing = _5_prime_nt + _3_prime_nt
        if _pairing in _unambiguous_snps.keys():
            if _5_prime_nt in ['A','T'] and _3_prime_nt not in ['A','T']:
                strand = 'TOP'
                allele_a = ref_allele
                allele_b = alt_allele
                return allele_a, allele_b, strand
            elif _3_prime_nt in ['A','T'] and _5_prime_nt not in ['A','T']:
                strand = 'BOT'
                allele_a = alt_allele
                allele_b = ref_allele
                return allele_a, allele_b, strand
            else:
                return seq, ref_allele, alt_allele, _5_prime_nt, _3_prime_nt

def assign_ab(allele_a=None, allele_b=None, strand=None):
    """
    Assign the AB genotype based on the nucleotide genotype and strand
    Args:
        allele_a: str, one of A, C, G, T
        allele_b: str, one of A, C, G, T
        strand: str, 

    Returns:
        tuple (allele_a, allele_b)
    """
    if allele_a is None:
        raise ValueError("allele_a should not be None")
    if allele_b is None:
        raise ValueError("allele_b should not be None")
    if strand is None:
        raise ValueError("strand should not be None")

    genotype = allele_a + allele_b

    if strand == 'TOP':
        if genotype == 'AA':
            return 'A','A'
        elif genotype == 'TT':
            return 'B','B'
        elif genotype == 'CC':
            return 'B','B'
        elif genotype == 'GG':
            return 'B','B'
        elif genotype == 'AT':
            return 'A','B'
        elif genotype == 'CG':
            return 'A','B'

    elif strand == 'BOT':
        if genotype == 'AA':
            return 'B','B'
        elif genotype == 'TT':
            return 'A','A'
        elif genotype == 'CC':
            return 'B','B'
        elif genotype == 'GG':
            return 'B','B'
        elif genotype == 'TA':
            return 'A','B'
        elif genotype == 'GC':
            return 'A','B'

    else:
        raise ValueError("Strand should be either TOP or BOT")


def convert_genotype(genotype, ref_allele, alt_allele):
    if genotype == '0/0':
        return ref_allele, ref_allele
    elif genotype == '1/1':
        return alt_allele, alt_allele
    elif genotype == '0/1':
        return ref_allele, alt_allele

    elif genotype == './.':
        return '-', '-'
    elif genotype == (None,None):
        return '-', '-'
    else:
        raise ValueError(f"Genotype {genotype} not recognized")

def tuple_to_float(x):
    if x == ('A','A'):
        return 0.
    elif x == ('A','B'):
        return 1.
    elif x == ('B','B'):
        return 2.
    else:
        return np.nan

In [ ]:
REF = '../../../REFERENCES/bosTau6/bosTau6_w_wagyuY.fa'
reference = Fasta(REF)
SAMPLES = '/Users/callummacphillamy/Projects/bovine_pangenome/TuWa_manuscript/metadata/25_animals_MCG.xlsx'
samples = pd.read_excel(SAMPLES)
samples.head()
sample_ids = samples['ID'].to_list()

sample_dict = {}
for sample in samples.itertuples():
    if sample.No < 10:
        sample_no = f'sample0{sample.No}'
    else:
        sample_no = f'sample{sample.No}'
    
    sample_dict[sample_no] = sample.ID

In [ ]:
snp_map = pd.read_csv('../../../GRIP/GRIP_data/Providers/3D_Genetics/Clients/3D_Genetics/Populations/3DG_Nucleus/Standard_Analyses/we_3D_pub_03022023_Versa50K_3/we_3D_pub_snp_map_03022023_Versa50K_3.txt',
                      sep='\t')
snp_map.head()
snp_map_dict = {}
for row in snp_map.itertuples():
    chr_pos = f'{row[3]}_{row[4]}'
    alleles = row.SNP.strip('[]').split('/')
    snp_map_dict[chr_pos] = {0.:f'{alleles[0]}{alleles[0]}',
                             1.:f'{alleles[0]}{alleles[1]}',
                             2.:f'{alleles[1]}{alleles[1]}'}
snp_map_dict['1_83267242']
snp_map[snp_map['Chromosome'] == '29']

In [ ]:
ARR = '../SNP_analyses/genotypes/3DG_Nucleus_array_genotypes.parquet'
arr = pd.read_parquet(ARR)
#arr[arr['Chr'] == '29']

In [ ]:
# Load the lifted over SNPs
#snps = pd.read_csv('../SNP_analyses/UMD3.1.1_to_ARS-UCD1.2.versa50K.bed', sep='\t', names=['chromosome','start','stop','UMD3.1_coords','-'])
snps = pd.read_csv('../SNP_analyses/SNP_bed_files/versa50K_SNPs.liftover.bed', sep='\t', names=['chromosome','start','stop'])
#snps = snps.iloc[:, :-1]
new_snps = snps['chromosome'].str.replace('chr','').str.cat(snps['stop'].astype(str), sep='_').to_list()
#new_snps = snps['chromosome'].str.replace('chr','').str.cat(snps['start'].astype(str), sep='_')
#new_snps_alt = snps['chromosome'].str.replace('chr','').str.cat(snps['stop'].astype(str), sep='_')
# #new_snps = new_snps.to_list()
#print(f'SNPs able to be lifted over: {len(new_snps)}')
#print(f'SNPs able to be lifted over: {len(new_snps_alt)}')
print(f'SNPs: ', len(new_snps))

In [ ]:
snps

In [ ]:
# # Convert SNPs for the Arr data to ARS-UCD1.2
# snp_umd_to_ars_dict = {}
# for snp in tqdm(snps.itertuples()):
#     umd_chrom, umd_snp = re.split(r'[:-]', snp[-1])[:2]
#     
#     umd_chrom = umd_chrom.replace('chr','')
#     umd_chrpos = f'{umd_chrom}_{umd_snp}'
#     
#     #print(umd_chrpos)
#     
#     ars_chrom = snp[1].replace('chr','')
#     ars_pos = snp[2]
#     
#     ars_chrpos = f'{ars_chrom}_{ars_pos}'
#     
#     snp_umd_to_ars_dict[umd_chrpos] = ars_chrpos
#     

In [ ]:
# ars_pos_col = []
# for chr_pos in arr['ChrPos']:
#     if chr_pos in snp_umd_to_ars_dict.keys():
#         ars_pos_col.append(snp_umd_to_ars_dict[chr_pos])
#     else:
#         ars_pos_col.append('0_0')
# arr.insert(4, 'ARS-UCD1.2_ChrPos', ars_pos_col)
# arr_50k = arr[arr['ARS-UCD1.2_ChrPos'] != '0_0']


In [ ]:
#arr_50k[arr_50k['Chr'] == '29']
arr

In [ ]:
# Load the SNP map for the 50k array
snp_map = pd.read_csv('/Users/callummacphillamy/Projects/GRIP/GRIP_data/Providers/3D_Genetics/Clients/3D_Genetics/Populations/3DG_Nucleus/Standard_Analyses/we_3D_pub_03022023_Versa50K_3/we_3D_pub_snp_map_03022023_Versa50K_3.txt', sep='\t')

snp_map_dict = {}
for snp in snp_map.itertuples():
    snp_map_dict[snp.Name] = snp.SNP
snp_map_dict['BovineHD0100000033']

In [ ]:
# Only run once
# AGGREGATE = '../SNP_analyses/reference/3DG_Nucleus-8685_137472-Aggregated_Genotype_Counts-2024_03_14_13_33_09.833399.tsv.gz'
# aggregate = pd.read_csv(AGGREGATE, sep='\t', comment='#', dtype={1:str,2:str})
# aggregate = aggregate[aggregate['ChrPos'] != '0_0']
# aggregate = aggregate[aggregate['Chr'] != '0']
# aggregate = aggregate[aggregate['ChrPos'].notna()]
# aggregate = aggregate[aggregate['Pos'].notna()]
# aggregate['Pos'] = aggregate['Pos'].astype(int)
# 
# aggregate_info = aggregate.iloc[:, :4]
# aggregate_samples = aggregate.loc[:, sample_ids]
# new_aggregate = pd.concat([aggregate_info, aggregate_samples], axis='columns')
# new_aggregate.to_parquet('../SNP_analyses/genotypes/3DG_Nucleus_array_genotypes.parquet')

In [ ]:
arr_50k = arr[arr['ChrPos'].isin(new_snps)]
arr_50k

In [ ]:
sample_dict

In [ ]:
ONT_SNPS = '../SNP_analyses/genotypes/mapped_to_bosTau6/sample12.versa50K.vcf.gz'
#_50k_snps = arr_50k['ARS-UCD1.2_ChrPos'].to_list()

def genotype_to_nuc_genotype(genotype, ref, alt):
    if genotype == ('.','.'):
        nuc_genotype = '--'
    else:
        genotype = tuple(map(int, genotype))
        alleles = list(ref) + list(alt)
        nuc_genotype = alleles[genotype[0]] + alleles[genotype[1]]
    return nuc_genotype

vcf = VariantFile(ONT_SNPS)
for record in tqdm(vcf.fetch()):
    samples = record.samples.keys()
    break
samples = list(samples)
overlap = 0

vcf = pd.read_csv(ONT_SNPS, sep='\t', comment='#',
                  header=None,
                  names=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT'] + samples)

ont_df_dict = {'ChrPos':[],
               'sample':[],
               'genotype':[],
               'allele_gt':[],
               'depth':[]}
samples = vcf.columns[9:]
samples = [sample_dict[sample] for sample in samples]
for row in tqdm(vcf.itertuples()):

    chr_pos = str(row.CHROM).replace('chr','') + '_' + str(row.POS)
    #if chr_pos in _50k_snps:
    ref = row.REF
    alt = row.ALT
    
    
    locus_genotypes = list(map(lambda gt: gt.split(':')[0], row[10:]))
    vcf_genotypes = list(map(lambda gt: tuple(gt.split('/')), locus_genotypes))     
    allele_genotypes = list(map(lambda gt: genotype_to_nuc_genotype(gt, ref, alt), vcf_genotypes))
    
    if row.INFO == '.':
        continue
    else:
        depth = int(row.INFO.split(';')[0].split('=')[1])
    
    if len(vcf_genotypes) != len(samples):
        print('Length of genotypes does not match length of samples')
        print(len(vcf_genotypes), len(samples))
        print(row)
        break
    
    ont_df_dict['ChrPos'].extend([chr_pos] * len(samples))
    ont_df_dict['sample'].extend(samples)
    ont_df_dict['genotype'].extend(vcf_genotypes)
    ont_df_dict['allele_gt'].extend(allele_genotypes)
    ont_df_dict['depth'].append(depth)
x = pd.DataFrame.from_dict(ont_df_dict)


In [ ]:
snp_map_for_conversion = snp_map[['Chromosome','Position','SNP']]
snp_map_for_conversion = snp_map_for_conversion.copy()
snp_map_for_conversion['SNP'] = snp_map_for_conversion['SNP'].str.strip('[]')
snp_map_for_conversion['ChrPos'] = snp_map_for_conversion['Chromosome'].astype(str).str.cat(snp_map_for_conversion['Position'].astype(str), sep='_')
snp_map_for_conversion['Allele_A'] = snp_map_for_conversion['SNP'].str.split('/').str[0]
snp_map_for_conversion['Allele_B'] = snp_map_for_conversion['SNP'].str.split('/').str[1]
snp_map_for_conversion

In [ ]:
new_x = pd.merge(x, snp_map_for_conversion[['ChrPos','Allele_A','Allele_B']], how='inner', left_on='ChrPos', right_on='ChrPos')
new_x_dict = {'ChrPos':[],
              'sample':[],
              'genotype':[],
              'allele_gt':[],
              'allele_A':[],
              'allele_B':[],
              'Illumina_gt':[],
              'depth':[]}

a_dict = {'5-7':new_x_dict,
          '8-9':new_x_dict,
          '10+':new_x_dict}


complementary_bases = {'A':'T',
                       'T':'A',
                       'C':'G',
                       'G':'C'}

for row in tqdm(new_x.itertuples()):
    
    _chrpos = row.ChrPos
    _sample = row.sample
    _genotype = row.genotype
    _allele_gt = row.allele_gt
    _allele_A = row.Allele_A
    _allele_B = row.Allele_B
    _depth = int(row.depth)
    
    allele_1 = row.allele_gt[0]
    allele_2 = row.allele_gt[1]
    
    if allele_1 == allele_2:
        if allele_1 == row.Allele_A:
            _ill_gt = 0.
            
        elif allele_1 == row.Allele_B:
            _ill_gt = 2.
        elif allele_1 == complementary_bases[row.Allele_A]:
            _ill_gt = 0.
        elif allele_1 == complementary_bases[row.Allele_B]:
            _ill_gt = 2.
        else:
           _ill_gt = np.nan
    
    elif allele_1 != allele_2:
        _ill_gt = 1.
    else:
        print('Something went wrong')
        break
    
    if _depth < 5:
        continue
    elif 5 <= _depth <= 7:
        a_dict['5-7']['ChrPos'].append(_chrpos)
        a_dict['5-7']['sample'].append(_sample)
        a_dict['5-7']['genotype'].append(_genotype)
        a_dict['5-7']['allele_gt'].append(_allele_gt)
        a_dict['5-7']['allele_A'].append(_allele_A)
        a_dict['5-7']['allele_B'].append(_allele_B)
        a_dict['5-7']['Illumina_gt'].append(_ill_gt)
        a_dict['5-7']['depth'].append(_depth)
        continue
    elif 8 <= _depth <= 9:
        a_dict['8-9']['ChrPos'].append(_chrpos)
        a_dict['8-9']['sample'].append(_sample)
        a_dict['8-9']['genotype'].append(_genotype)
        a_dict['8-9']['allele_gt'].append(_allele_gt)
        a_dict['8-9']['allele_A'].append(_allele_A)
        a_dict['8-9']['allele_B'].append(_allele_B)
        a_dict['8-9']['Illumina_gt'].append(_ill_gt)
        a_dict['8-9']['depth'].append(_depth)
        continue
    elif _depth >= 10:
        a_dict['10+']['ChrPos'].append(_chrpos)
        a_dict['10+']['sample'].append(_sample)
        a_dict['10+']['genotype'].append(_genotype)
        a_dict['10+']['allele_gt'].append(_allele_gt)
        a_dict['10+']['allele_A'].append(_allele_A)
        a_dict['10+']['allele_B'].append(_allele_B)
        a_dict['10+']['Illumina_gt'].append(_ill_gt)
        a_dict['10+']['depth'].append(_depth)
        continue
    
a = pd.DataFrame.from_dict(a_dict['10+'])
a = a[a['depth'] >= 10]
a

In [ ]:
y = arr[['ChrPos', '3DWF22T0240']]
y = y.copy()
z = pd.merge(a[['ChrPos','Illumina_gt']], y, how='inner', left_on='ChrPos', right_on='ChrPos')
results_dict = {'Sample':[],
                'missing_in_ont':[],
                'missing_in_arr':[],
                'missing_in_both':[],
                'correct':[],
                'half_correct':[],
                'incorrect':[]}
missing_in_ont = 0
missing_in_arr = 0
missing_in_both = 0
correct = 0
half_correct = []
incorrect = []
comp_dict = {'AA':'TT',
             'TT':'AA',
             'CC':'GG',
             'GG':'CC',
             'AT':'TA',
             'TA':'AT',
             'CG':'GC',
             'GC':'CG'}
comparable_records = 0
for row in z.itertuples():
    
    
    if np.isnan(row.Illumina_gt) and np.isnan(row._3):
        missing_in_both += 1
        continue
    elif np.isnan(row.Illumina_gt):
        missing_in_ont += 1
        continue
    elif np.isnan(row._3):
        missing_in_arr += 1
        continue  
    
    if row.Illumina_gt == row._3:
        correct += 1
        comparable_records += 1
    elif np.abs(row.Illumina_gt - row._3) == 2:
        incorrect.append(row)
        comparable_records += 1
    elif np.abs(row.Illumina_gt - row._3) == 1:
        half_correct.append(row)
        comparable_records += 1
    
    
print(f'Missing in both: {missing_in_both}')
print(f'Missing in ont: {missing_in_ont}')
print(f'Missing in arr: {missing_in_arr}')
print(f'Correct: {correct}')
print(f'Half correct: {len(half_correct)}')
print(f'Incorrect: {len(incorrect)}')
print('Comparable records:', comparable_records)
print('Concordance rate:', (correct / comparable_records))
# print(f'Concordance rate: {(correct + len(half_correct)) / ((correct + len(incorrect)) + len(half_correct))}')

In [ ]:
# y = arr_50k[['ChrPos','ARS-UCD1.2_ChrPos','3DWF22T0415']]
# y = y.copy()
# y = y[y['ARS-UCD1.2_ChrPos'].isin(x['ChrPos'])]
# z = pd.merge(x, y, how='inner', left_on='ChrPos', right_on='ARS-UCD1.2_ChrPos')
# missing_in_ont = 0
# missing_in_arr = 0
# missing_in_both = 0
# correct = 0
# incorrect = []
# for row in z.itertuples():
#     
#     if np.isnan(row._7) and row.allele_gt == '--':
#         missing_in_both += 1
#         continue
#     elif row.allele_gt == '--':
#         missing_in_ont += 1
#         continue
#     elif np.isnan(row._7):
#         missing_in_arr += 1
#         continue  
#     
#     arr_nuc_gt = snp_map_dict[row.ChrPos_y][row._7]
#     
#     if arr_nuc_gt == row.allele_gt:
#         correct += 1
#     elif arr_nuc_gt != row.allele_gt:
#         incorrect.append(row)
# print(f'Missing in both: {missing_in_both}')
# print(f'Missing in ont: {missing_in_ont}')
# print(f'Missing in arr: {missing_in_arr}')
# print(f'Correct: {correct}')
# #print(f'Incorrect: {incorrect}')


In [ ]:
# ONT_SNPS = '../SNP_analyses/genotypes/ont_array_comparisons/test_17_samples_29_versa50K.vcf.gz'
# _50k_snps = arr_50k['ARS-UCD1.2_ChrPos'].to_list()
# 
# vcf = VariantFile(ONT_SNPS)
# for record in tqdm(vcf.fetch()):
#     samples = record.samples.keys()
#     break
# samples = list(samples)
# overlap = 0
# overlap_1 = 0
# overlap_2 = 0
# 
# vcf = pd.read_csv(ONT_SNPS, sep='\t', comment='#',
#                   header=None,
#                   names=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT'] + samples)
# #vcf['ChrPos'] = vcf['CHROM'].astype(str).str.cat(vcf['POS'].astype(str), sep='_')
# 
# possibly_repetitive_region = []
# indels = []
# multiallelic = []
# possibly_wrong_coordinate = {}
# cannot_find_alt = {}
# new_records = []
# 
# 
# for row in tqdm(vcf.itertuples()):
#     
#     if 'INDEL' in row.INFO:
#         indels.append(row)
#         
#         continue
#     if len(row.REF) > 1:
#         indels.append(row)
#         continue
#     chr_pos = "chr" + str(row.CHROM) + "_" + str(row.POS)
#     if len(row.ALT) > 1 and row.ALT != '<NON_REF>':
#         multiallelic.append(row)
#         continue
#     try:
#         ref_allele = dbsnp_dict[chr_pos]['Ref']
#         alt_allele = dbsnp_dict[chr_pos]['Alt']
#     except KeyError:
#         possibly_wrong_coordinate[chr_pos] = (row.CHROM, row.POS)
#         continue
# 
#     locus_genotypes = row[10:]
#     locus_info = row[1:10]
#     locus_genotypes = [gt.split(':')[0] for gt in locus_genotypes]
#     # print(locus_info)
#     # #print(locus_genotypes)
#     # print(allele_freq)
#     # print(locus_genotypes)
#     # print(locus_info)
#     # print(ref_allele)
#     # print(alt_allele)
#     # break
#     sample_genotypes = list(map(lambda gt: convert_genotype(gt, ref_allele, alt_allele), locus_genotypes))
#     
#     if alt_allele is not None:
#         snp = ref_allele + alt_allele
#         if snp in unambiguous_snps.keys():
#             allele_a, allele_b, strand = unambiguous_snps[snp]
#             ab_genotype = list(map(lambda gt: assign_ab(gt[0], gt[1], strand), sample_genotypes))
#             float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
#         elif snp not in unambiguous_snps.keys():
#             try:
#                 seq = get_sequence(reference, (str(row.CHROM), row.POS), ref_allele)
#             except ValueError:
#                 ref_allele = row.ALT
#                 alt_allele = row.REF
#                 seq = get_sequence(reference, (str(row.CHROM), row.POS), ref_allele)
#             allele_a, allele_b, strand = resolve_ambiguous_snp(seq, ref_allele, alt_allele)
#             ab_genotype = list(map(lambda gt: assign_ab(gt[0], gt[1], strand), sample_genotypes))
#             float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
#             #print(row)
#     elif alt_allele is None:
#         raise ValueError("Alt allele is None")
#         #print(row)
#         # allele_a, strand = resolve_ambiguous_snp_single_allele(seq, ref_allele)
#         # ab_genotype = list(map(lambda gt: assign_ab_single_allele(gt[0], strand), sample_genotypes))
#         # float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
#         #print(ab_genotype)
#         break
#     
#     else:
#         print('Something unforeseen happened')  
#         break
#     new_record = list(locus_info)
#     new_record.extend(float_genotype)
#     new_records.append(new_record)

In [ ]:
ONT_SNPS = glob.glob('../RESULTS/batch*_results/variants/SNPs/sample*/*/merge_output.vcf.gz')
ONT_SNPS = [ONT_SNPS[12]]
for ont_snp in ONT_SNPS:
    if 'MT' in ont_snp:
        continue
    else:
        sample = sample_dict[ont_snp.split('/')[5]]
        x = load_vcf(ont_snp, sample)
        x['ChrPos'] = x['CHROM'].astype(str).str.cat(x['POS'].astype(str), sep='_')
        # possibly_repetitive_region = []
        # indels = []
        # multiallelic = []
        # possibly_wrong_coordinate = {}
        # cannot_find_alt = {}
        # 
        # new_records = []
        # 
        # for row in tqdm(x.itertuples()):
        #     if 'INDEL' in row.INFO:
        #         indels.append(row)
        #         continue
        #     if len(row.REF) > 1:
        #         indels.append(row)
        #         continue
        #     chr_pos = str(row.CHROM) + "_" + str(row.POS)
        #     if len(row.ALT) > 1:
        #         multiallelic.append(row)
        #         continue
        #     # try:
        #         # Get the information for this SNP from the regions file
        #         # snp_name = regions.loc[regions['ChrPos'] == chr_pos, 'Name'].values[0]
        #     # except IndexError:
        #     #     print('Couldn\'t determine SNP name from manifest. Using chr_pos_ref as snp name')
        #     #     snp_name = str(row.CHROM) + "_" + str(row.POS) + "_" + reference_name
        #     #     possibly_wrong_coordinate[snp_name] = (row.CHROM, row.POS)
        #     
        #     # allele_freq = allele_frequencies.loc[
        #     #     allele_frequencies['ChrPos'] == f'chr{chr_pos}', '{ALLELE:FREQ}'
        #     # ].values[0]
        #     # ref_allele, ref_freq = allele_freq.split(';')[0].split(':')
        #     # if len(ref_allele) > 1:
        #     #     continue
        #     # try:
        #     #     alt_allele, alt_freq = allele_freq.split(';')[1].split(':')
        #     # except IndexError:
        #     #     try:
        #     #         alt_allele = dbsnp_dict[row.CHROM + "_" + str(row.POS)]['Alt']
        #     #     except KeyError:
        #     #         possibly_wrong_coordinate[snp_name] = (row.CHROM, row.POS)
        #     #         cannot_find_alt[chr_pos] = None
        #     #         continue
        # 
        #     if len(row.ALT) > 1:
        #         multiallelic.append(row)
        #         continue
        #     
        #     ref_allele = row.REF
        #     alt_allele = row.ALT[0]
        #     
        #     locus_info = row[:10]
        #     locus_genotype = row[-2].split(':')[0]
        #     sample_genotype = convert_genotype(locus_genotype, ref_allele, alt_allele)
        #     #print(sample_genotype)
        #     # locus_genotypes = [gt.split(':')[0] for gt in locus_genotypes]
        #     # print(locus_info)
        #     # #print(locus_genotypes)
        #     # print(allele_freq)
        #     # sample_genotypes = list(map(lambda gt: convert_genotype(gt, ref_allele, alt_allele), locus_genotypes))
        #     
        #     # print(sample_genotypes)
        #     #print(row.CHROM, row.POS, ref_allele)
        #     seq = get_sequence(reference, (row.CHROM, row.POS), ref_allele)
        #     if alt_allele is not None:
        #         snp = ref_allele + alt_allele
        #         if snp in unambiguous_snps.keys():
        #             allele_a, allele_b, strand = unambiguous_snps[snp]
        #             #ab_genotype = list(map(lambda gt: assign_ab(gt[0], gt[1], strand), sample_genotypes))
        #             ab_genotype = assign_ab(sample_genotype[0], sample_genotype[1], strand)
        #             #float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
        #             float_genotype = tuple_to_float(ab_genotype)
        #         elif snp not in unambiguous_snps.keys():
        #             try:
        #                 allele_a, allele_b, strand = resolve_ambiguous_snp(seq, ref_allele, alt_allele)
        #                 #ab_genotype = list(map(lambda gt: assign_ab(gt[0], gt[1], strand), sample_genotypes))
        #                 ab_genotype = assign_ab(sample_genotype[0], sample_genotype[1], strand)
        #                 #float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
        #                 float_genotype = tuple_to_float(ab_genotype)
        #                 #print(row)
        #             except TypeError:
        #                 possibly_repetitive_region.append(row)
        #                 continue
        #     elif alt_allele is None:
        #         raise ValueError("Alt allele is None")
        #         #print(row)
        #         # allele_a, strand = resolve_ambiguous_snp_single_allele(seq, ref_allele)
        #         # ab_genotype = list(map(lambda gt: assign_ab_single_allele(gt[0], strand), sample_genotypes))
        #         # float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
        #         #print(ab_genotype)
        #         break
        #     
        #     else:
        #         print('Something unforeseen happened')  
        #         break
        #     new_record = list(locus_info)
        #     new_record.append(float_genotype)
        #     new_records.append(new_record)
        #     
        #     ont_as_ab = pd.DataFrame.from_dict(new_records)
        #     ont_as_ab = ont_as_ab.iloc[:, 1:]
        #     ont_as_ab.columns = x.columns.to_list()[:-1]
        #     ont_as_ab['ChrPos'] = ont_as_ab['CHROM'].astype(str).str.cat(ont_as_ab['POS'].astype(str), sep='_')
        #     arr_sample = arr.loc[:,['ARS-UCD1.2_ChrPos', sample]]
        #     y = pd.merge(ont_as_ab, arr_sample, how='inner', left_on='ChrPos', right_on='ARS-UCD1.2_ChrPos')
        #     y.to_csv('../SNP_analyses/genotypes/ont_array_comparisons/{}.csv'.format(sample))

In [ ]:
common_snps = pd.read_csv('../SNP_analyses/common_SNPs.bed', sep='\t',
                          header=0)
common_snps['ChrPos'] = common_snps['#chrom'].astype(str).str.cat(common_snps['start_(snpPos)'].astype(str), sep='_')
common_snps

In [ ]:
possibly_repetitive_region = []
indels = []
multiallelic = []
possibly_wrong_coordinate = {}
cannot_find_alt = {}

new_records = []

common_snp_list = {chrpos:None for chrpos in common_snps['ChrPos']}
common_snp_list_1 = {}
for row in common_snps.itertuples():
    common_snp_list_1[f"{row[1]}_{row[3]}"] = None
for row in tqdm(x.itertuples()):
    if row.ChrPos not in common_snp_list.keys():
        continue
    # elif row.ChrPos not in common_snp_list_1.keys():
    #     continue
    if 'INDEL' in row.INFO:
        indels.append(row)
        continue
    if len(row.REF) > 1:
        indels.append(row)
        continue
    chr_pos = str(row.CHROM) + "_" + str(row.POS)
    if len(row.ALT) > 1:
        multiallelic.append(row)
        continue
    # try:
        # Get the information for this SNP from the regions file
        # snp_name = regions.loc[regions['ChrPos'] == chr_pos, 'Name'].values[0]
    # except IndexError:
    #     print('Couldn\'t determine SNP name from manifest. Using chr_pos_ref as snp name')
    #     snp_name = str(row.CHROM) + "_" + str(row.POS) + "_" + reference_name
    #     possibly_wrong_coordinate[snp_name] = (row.CHROM, row.POS)
    
    # allele_freq = allele_frequencies.loc[
    #     allele_frequencies['ChrPos'] == f'chr{chr_pos}', '{ALLELE:FREQ}'
    # ].values[0]
    # ref_allele, ref_freq = allele_freq.split(';')[0].split(':')
    # if len(ref_allele) > 1:
    #     continue
    # try:
    #     alt_allele, alt_freq = allele_freq.split(';')[1].split(':')
    # except IndexError:
    #     try:
    #         alt_allele = dbsnp_dict[row.CHROM + "_" + str(row.POS)]['Alt']
    #     except KeyError:
    #         possibly_wrong_coordinate[snp_name] = (row.CHROM, row.POS)
    #         cannot_find_alt[chr_pos] = None
    #         continue

    if len(row.ALT) > 1:
        multiallelic.append(row)
        continue
    
    ref_allele = row.REF
    alt_allele = row.ALT[0]
    
    locus_info = row[:10]
    locus_genotype = row[-2].split(':')[0]
    sample_genotype = convert_genotype(locus_genotype, ref_allele, alt_allele)
    #print(sample_genotype)
    # locus_genotypes = [gt.split(':')[0] for gt in locus_genotypes]
    # print(locus_info)
    # #print(locus_genotypes)
    # print(allele_freq)
    # sample_genotypes = list(map(lambda gt: convert_genotype(gt, ref_allele, alt_allele), locus_genotypes))
    
    # print(sample_genotypes)
    #print(row.CHROM, row.POS, ref_allele)
    seq = get_sequence(reference, (row.CHROM, row.POS), ref_allele)
    if alt_allele is not None:
        snp = ref_allele + alt_allele
        if snp in unambiguous_snps.keys():
            allele_a, allele_b, strand = unambiguous_snps[snp]
            #ab_genotype = list(map(lambda gt: assign_ab(gt[0], gt[1], strand), sample_genotypes))
            ab_genotype = assign_ab(sample_genotype[0], sample_genotype[1], strand)
            #float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
            float_genotype = tuple_to_float(ab_genotype)
        elif snp not in unambiguous_snps.keys():
            try:
                allele_a, allele_b, strand = resolve_ambiguous_snp(seq, ref_allele, alt_allele)
                #ab_genotype = list(map(lambda gt: assign_ab(gt[0], gt[1], strand), sample_genotypes))
                ab_genotype = assign_ab(sample_genotype[0], sample_genotype[1], strand)
                #float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
                float_genotype = tuple_to_float(ab_genotype)
                #print(row)
            except TypeError:
                possibly_repetitive_region.append(row)
                continue
    elif alt_allele is None:
        raise ValueError("Alt allele is None")
        #print(row)
        # allele_a, strand = resolve_ambiguous_snp_single_allele(seq, ref_allele)
        # ab_genotype = list(map(lambda gt: assign_ab_single_allele(gt[0], strand), sample_genotypes))
        # float_genotype = list(map(lambda x: tuple_to_float(x), ab_genotype))
        #print(ab_genotype)
        break
    
    else:
        print('Something unforeseen happened')  
        break
    new_record = list(locus_info)
    new_record.append(float_genotype)
    new_records.append(new_record)
    
    ont_as_ab = pd.DataFrame.from_dict(new_records)
    ont_as_ab = ont_as_ab.iloc[:, 1:]
    ont_as_ab.columns = x.columns.to_list()[:-1]
    ont_as_ab['ChrPos'] = ont_as_ab['CHROM'].astype(str).str.cat(ont_as_ab['POS'].astype(str), sep='_')
    arr_sample = arr.loc[:,['ARS-UCD1.2_ChrPos', sample]]
    y = pd.merge(ont_as_ab, arr_sample, how='inner', left_on='ChrPos', right_on='ARS-UCD1.2_ChrPos')
    #y.to_csv('../SNP_analyses/genotypes/ont_array_comparisons/{}.csv'.format(sample))

In [ ]:
ont_as_ab = pd.DataFrame.from_dict(new_records)
ont_as_ab = ont_as_ab.iloc[:, 1:]
ont_as_ab.columns = x.columns.to_list()[:-1]
ont_as_ab['ChrPos'] = ont_as_ab['CHROM'].astype(str).str.cat(ont_as_ab['POS'].astype(str), sep='_')
ont_as_ab

In [ ]:
row

In [ ]:
arr_sample = arr.loc[:,['ARS-UCD1.2_ChrPos', sample]]
arr_sample

In [ ]:
y = pd.merge(ont_as_ab, arr_sample, how='inner', left_on='ChrPos', right_on='ARS-UCD1.2_ChrPos')
y

In [ ]:
ref_allele

In [ ]:
#`~/software/gatk-4.4.0.0/gatk GenotypeGVCFs -R ~/Projects/REFERENCES/bosTau6/bosTau6.fa -V merge_output.gvcf.gz -O test.vcf.gz -L test.bed -all-sites`